# ELMo

In [1]:
import pandas as pd
import numpy as np
import spacy
import re
import time
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0604 14:26:13.054023 11724 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [2]:
pd.set_option('display.max_colwidth', 200)
tf.logging.set_verbosity(tf.logging.WARN)

In [3]:
data_dir = "data"

## Load IMDB Data

In [4]:
imdb_data = pd.read_pickle(f"{data_dir}/imdb_data.pickle.gz")

In [5]:
imdb_data.sample(2)

,data_set,polarity,sentence,movie_id
13640,test,1,I watched this mini in the early eighties. Sam Waterson proved himself to be a great actor. In fact when he began Law and Order I was disappointed in him as it was not as powerful a role. Unfortun...,tt0163955
18916,test,1,If you like silly comedies like Airplane you'll love this movie! It's definitely in the style of Airplane and Scary Movie. A fun film! It has the strangest cast of characters all in the same movie...,tt0395669


In [6]:
imdb_data = imdb_data.sample(800)

## Cleaning Text

In [7]:
max_words = 128

text_http_re  = re.compile(r'http\S+')
text_digit_re = re.compile(r'[0-9]')
text_html_re  = re.compile(r'<[^>]{0,20}>')
text_punc_re  = re.compile('[' + re.escape('\'!"#$%&()*+-/:;<=>?@[\\]^_`{|}~') + ']')
text_ws_re    = re.compile('\s+')

def clean_text(text):
    text = text.lower()
    text = text_http_re.sub('', text)
    text = text_html_re.sub('', text)
    text = text_digit_re.sub(' ', text)
    text = text_punc_re.sub('', text)
    text = text_ws_re.sub(' ', text)
    text = text.strip()
    return text

def create_lemmatizer_spacy():
    nlp = spacy.load('en', disable=['parser', 'ner'])
    def lemmatize(text):
        return ' '.join([token.lemma_ for token in nlp(text)][0:max_words])
    
    return lemmatize

def create_lemmatizer_nltk():
    from nltk.stem import WordNetLemmatizer 
    lemmatizer = WordNetLemmatizer()
    
    def lemmatize(text):
        return ' '.join([lemmatizer.lemmatize(w) for w in text.split()][0:max_words])
    
    return lemmatize

# Setup a lemmatize function, spacy.load may fail on windows for en.
try:
    lemmatize = create_lemmatizer_spacy()
except:
    print("Using nltk for lemmatization.")
    lemmatize = create_lemmatizer_nltk()
            
def process_text(text):
    return lemmatize(clean_text(text))

Using nltk for lemmatization.


In [8]:
imdb_data['clean_review'] = imdb_data.sentence.apply(process_text)

## Extract ELMo Embeddings

In [9]:
# Utility function to break sentences into batches.
def batches(sentences, batch_size):
    results = []
    chunk = []
    for s in sentences:
        chunk.append(s)
        if len(chunk) >= batch_size:
            yield(chunk)
            chunk = []
    if len(chunk) > 0:
        yield(chunk)

def elmo_create_embedding_extractor(module, batch_size=20):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module, trainable=True)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
        
    def extract(_sentences):
        results = []
        for s_batch in batches(_sentences, batch_size):
            results.extend(session.run(embeddings, { sentences: s_batch }))
            print(f"[{datetime.now()}] Extracted {len(results)}")
        return results

    return extract

elmo_get_embedding = elmo_create_embedding_extractor("https://tfhub.dev/google/elmo/2", 50)

Instructions for updating:
Colocations handled automatically by placer.


W0604 14:27:35.636558 11724 deprecation.py:323] From c:\users\bryanc\source\repos\mlads-textembedding-bert-elmo-tutorial\venv\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [10]:
current_time = datetime.now()
imdb_data['embedding'] = elmo_get_embedding(imdb_data.clean_review.values)
print("Extraction took time ", datetime.now() - current_time)

[2019-06-04 14:28:55.533996] Extracted 50
[2019-06-04 14:29:47.236447] Extracted 100
[2019-06-04 14:30:37.786700] Extracted 150
[2019-06-04 14:31:28.215075] Extracted 200
[2019-06-04 14:32:20.365033] Extracted 250
[2019-06-04 14:33:11.260297] Extracted 300
[2019-06-04 14:34:02.972305] Extracted 350
[2019-06-04 14:34:54.654950] Extracted 400
[2019-06-04 14:35:46.315738] Extracted 450
[2019-06-04 14:36:36.021616] Extracted 500
[2019-06-04 14:37:26.341114] Extracted 550
[2019-06-04 14:38:19.087696] Extracted 600
[2019-06-04 14:39:11.035528] Extracted 650
[2019-06-04 14:40:01.452119] Extracted 700
[2019-06-04 14:40:52.376668] Extracted 750
[2019-06-04 14:41:44.998869] Extracted 800
Extraction took time  0:13:41.448381


In [11]:
imdb_data.to_pickle(f"{data_dir}/imdb_data_w_elmo_embedding.pickle.gz")